### Introduction
This notebook presents a RAG workflow for the [PubMed QA](https://pubmedqa.github.io/) task using [LlamaIndex](https://www.llamaindex.ai/). The code is written in a configurable fashion, giving you the flexibility to edit the RAG configuration and observe the change in output/responses.

It covers a step-by-step procedure for building the RAG workflow (Stages 1-4) and later runs the pipeline on a sample from the dataset. The notebook also covers the sparse, dense, hybrid retrieval strategies along with the re-ranker. We have alse added an optional component for RAG evaluation using the [Ragas](https://docs.ragas.io/en/stable/getstarted/install.html) library.

### Import libraries, custom classes and functions

In [1]:
from pathlib import Path
from pprint import pprint
import sys
import os
import random

from llama_index.core import ServiceContext, set_global_service_context, set_global_handler
from llama_index.core.node_parser import SentenceSplitter

from task_dataset import PubMedQATaskDataset

from utils.hosting_utils import RAGLLM
from utils.rag_utils import (
    DocumentReader, RAGEmbedding, RAGQueryEngine, RagasEval, 
    extract_yes_no, evaluate, validate_rag_cfg
    )
from utils.storage_utils import RAGIndex

In [2]:
import warnings
warnings.filterwarnings('ignore')

### Set RAG configuration

In [3]:
rag_cfg = {
    # Node parser config
    "chunk_size": 256,
    "chunk_overlap": 0,

    # Embedding model config
    "embed_model_type": "hf",
    "embed_model_name": "BAAI/bge-base-en-v1.5",

    # LLM config
    "llm_type": "local",
    "llm_name": "Llama-2-7b-chat-hf",
    "max_new_tokens": 256,
    "temperature": 0.0,
    "top_p": 1.0,
    "top_k": 50,
    "do_sample": False,

    # Vector DB config
    "vector_db_type": "weaviate", # "weaviate"
    "vector_db_name": "Pubmed_QA",
    
    # MODIFY THIS
    #"weaviate_url": "https://rag-bootcamp-pubmed-qa-n3u138r8.weaviate.network",
    "weaviate_url": "https://vector-rag-bootcamp-fqe3dp7f.weaviate.network",

    # Retriever and query config
    "retriever_type": "vector_index", # "vector_index"
    "retriever_similarity_top_k": 5,
    "query_mode": "hybrid", # "default", "hybrid"
    "hybrid_search_alpha": 0.0, # float from 0.0 (sparse search - bm25) to 1.0 (vector search)
    "response_mode": "compact",
    "use_reranker": False,
    "rerank_top_k": 3,

    # Evaluation config
    "eval_llm_type": "openai",
    "eval_llm_name": "gpt-3.5-turbo",
}

### Read secrets

#### Weaviate Key

In [4]:
try:
    f = open(Path.home() / ".weaviate.key", "r")
    f.close()
except Exception as err:
    print(f"Could not read your Weaviate key. Please make sure this is available in plain text under your home directory in ~/.weaviate.key: {err}")

#### Cohere API Key (required for re-ranker)

In [5]:
try:
    f = open(Path.home() / ".cohere.key", "r")
    os.environ["COHERE_API_KEY"] = f.read().rstrip("\n")
    f.close()
except Exception as err:
    print(f"Could not read your Cohere API key. Please make sure this is available in plain text under your home directory in ~/.cohere.key: {err}")

#### OpenAI API Key [Optional]

In [6]:
try:
    f = open(Path.home() / ".openai.key", "r")
    os.environ["OPENAI_API_KEY"] = f.read().rstrip("\n")
    f.close()
except Exception as err:
    print(f"Could not read your OpenAI API key. If you wish to run RAG evaluation, please make sure this is available in plain text under your home directory in ~/.openai.key: {err}")

## STAGE 0 - Preliminary config checks

In [7]:
validate_rag_cfg(rag_cfg)
pprint(rag_cfg)

{'chunk_overlap': 0,
 'chunk_size': 256,
 'do_sample': False,
 'embed_model_name': 'BAAI/bge-base-en-v1.5',
 'embed_model_type': 'hf',
 'eval_llm_name': 'gpt-3.5-turbo',
 'eval_llm_type': 'openai',
 'hybrid_search_alpha': 0.0,
 'llm_name': 'Llama-2-7b-chat-hf',
 'llm_type': 'local',
 'max_new_tokens': 256,
 'query_mode': 'hybrid',
 'rerank_top_k': 3,
 'response_mode': 'compact',
 'retriever_similarity_top_k': 5,
 'retriever_type': 'vector_index',
 'temperature': 0.0,
 'top_k': 50,
 'top_p': 1.0,
 'use_reranker': False,
 'vector_db_name': 'Pubmed_QA',
 'vector_db_type': 'weaviate',
 'weaviate_url': 'https://vector-rag-bootcamp-fqe3dp7f.weaviate.network'}


## STAGE 1 - Load dataset and documents

#### 1. Load PubMed QA dataset
PubMedQA ([github](https://github.com/pubmedqa/pubmedqa)) is a biomedical question answering dataset. Each instance consists of a question, a context (extracted from PubMed abstracts), a long answer and a yes/no/maybe answer. We make use of the test split of [this](https://huggingface.co/datasets/bigbio/pubmed_qa) huggingface dataset for this notebook.

**The context for each instance is stored as a text file** (referred to as documents), to align the task as a standard RAG use-case.

In [8]:
print('Loading PubMed QA data ...')
pubmed_data = PubMedQATaskDataset('bigbio/pubmed_qa')
print(f"Loaded data size: {len(pubmed_data)}")
pubmed_data.mock_knowledge_base(output_dir='./data', one_file_per_sample=True)

Loading PubMed QA data ...


Generating train split:   0%|          | 0/450 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/50 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/450 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/50 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/500 [00:00<?, ? examples/s]

Preparing data: 100%|███████████████████████| 500/500 [00:00<00:00, 1158.99it/s]


Loaded data size: 500


#### 2. Load documents
All metadata is excluded by default. Set the *exclude_llm_metadata_keys* and *exclude_embed_metadata_keys* flags to *false* for including it. Please refer to [this](https://docs.llamaindex.ai/en/stable/module_guides/loading/documents_and_nodes/usage_documents.html) and the *DocumentReader* class from *rag_utils.py* for further details.

In [9]:
print('Loading documents ...')
reader = DocumentReader(input_dir="./data/pubmed_doc")
docs = reader.load_data()
print(f'No. of documents loaded: {len(docs)}')

Loading documents ...
No. of documents loaded: 500


## STAGE 2 - Load node parser, embedding, LLM and set service context

#### 1. Load node parser to split documents into smaller chunks

In [10]:
print('Loading node parser ...')
node_parser = SentenceSplitter(chunk_size=rag_cfg['chunk_size'], chunk_overlap=rag_cfg['chunk_overlap'])
# nodes = node_parser.get_nodes_from_documents(docs)

Loading node parser ...


#### 2. Load embedding model
LlamaIndex supports embedding models from OpenAI, Cohere, HuggingFace, etc. Please refer to [this](https://docs.llamaindex.ai/en/stable/module_guides/models/embeddings.html#custom-embedding-model) for building a custom embedding model.

In [11]:
embed_model = RAGEmbedding(model_type=rag_cfg['embed_model_type'], model_name=rag_cfg['embed_model_name']).load_model()

Loading hf embedding model ...


config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

#### 3. Load LLM for generation
LlamaIndex supports LLMs from OpenAI, Cohere, HuggingFace, AI21, etc. Please refer to [this](https://docs.llamaindex.ai/en/stable/module_guides/models/llms/usage_custom.html#example-using-a-custom-llm-model-advanced) for loading a custom LLM model for generation.

In [12]:
llm = RAGLLM(rag_cfg['llm_type'], rag_cfg['llm_name']).load_model(**rag_cfg)

Loading local LLM model ...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

#### 4. Use service context to set the node parser, embedding model, LLM, etc.

In [13]:
service_context = ServiceContext.from_defaults(
    node_parser=node_parser,
    embed_model=embed_model,
    llm=llm,
)
# Set it globally to avoid passing it to every class, this sets it even for rag_utils.py
set_global_service_context(service_context)

## STAGE 3 - Create index using the appropriate vector store
All vector stores supported by LlamaIndex along with their available features are listed [here](https://docs.llamaindex.ai/en/stable/module_guides/storing/vector_stores.html).

If you are using LangChain, the supported vector stores can be found [here](https://python.langchain.com/docs/modules/data_connection/vectorstores/).

In [14]:
index = RAGIndex(db_type=rag_cfg['vector_db_type'], db_name=rag_cfg['vector_db_name'])\
    .create_index(docs, weaviate_url=rag_cfg["weaviate_url"])

Creating new index ...


## STAGE 4 - Build query engine

Now build a query engine using *retriever* and *response_synthesizer*. LlamaIndex also supports different types of [retrievers](https://docs.llamaindex.ai/en/stable/api_reference/query/retrievers.html) and [response modes](https://docs.llamaindex.ai/en/stable/module_guides/querying/response_synthesizers/root.html#configuring-the-response-mode) for various use-cases.

[Weaviate hybrid search](https://weaviate.io/blog/hybrid-search-explained) explains how dense and sparse search is combined.

In [15]:
def set_query_engine_args(rag_cfg, docs):
    query_engine_args = {
        "similarity_top_k": rag_cfg['retriever_similarity_top_k'], 
        "response_mode": rag_cfg['response_mode'],
        "use_reranker": False,
    }
    
    if (rag_cfg["retriever_type"] == "vector_index") and (rag_cfg["vector_db_type"] == "weaviate"):
        query_engine_args.update({
            "query_mode": rag_cfg["query_mode"], 
            "hybrid_search_alpha": rag_cfg["hybrid_search_alpha"]
        })
    elif rag_cfg["retriever_type"] == "bm25":
        nodes = service_context.node_parser.get_nodes_from_documents(docs)
        tokenizer = service_context.embed_model._tokenizer
        query_engine_args.update({"nodes": nodes, "tokenizer": tokenizer})
        
    if rag_cfg["use_reranker"]:
        query_engine_args.update({"use_reranker": True, "rerank_top_k": rag_cfg["rerank_top_k"]})

    return query_engine_args

In [16]:
query_engine_args = set_query_engine_args(rag_cfg, docs)
pprint(query_engine_args)

{'hybrid_search_alpha': 0.0,
 'query_mode': 'hybrid',
 'response_mode': 'compact',
 'similarity_top_k': 5,
 'use_reranker': False}


In [17]:
query_engine = RAGQueryEngine(
    retriever_type=rag_cfg['retriever_type'], vector_index=index, llm_model_name=rag_cfg['llm_name']).create(**query_engine_args)

## STAGE 5 - Finally query the model!
**Note:** We are using keyword based search or sparse search since *hybrid_search_alpha* is set to 0.0 by default.

#### [TODO] Change seed to experiment with a different sample

In [18]:
random.seed(237)

In [19]:
sample_idx = random.randint(0, len(pubmed_data)-1)
sample_elm = pubmed_data[sample_idx]
pprint(sample_elm)

{'answer': ['no'],
 'context': 'Human immunodeficiency virus (HIV)-infected patients have '
            'generally been excluded from transplantation. Recent advances in '
            'the management and prognosis of these patients suggest that this '
            'policy should be reevaluated. To explore the current views of '
            'U.S. transplant centers toward transplanting asymptomatic '
            'HIV-infected patients with end-stage renal disease, a written '
            'survey was mailed to the directors of transplantation at all 248 '
            'renal transplant centers in the United States. All 148 responding '
            'centers said they require HIV testing of prospective kidney '
            'recipients, and 84% of these centers would not transplant an '
            'individual who refuses HIV testing. The vast majority of '
            'responding centers would not transplant a kidney from a cadaveric '
            '(88%) or a living donor (91%) into an asymp

In [20]:
query = sample_elm['question']

response = query_engine.query(query)

print(f'QUERY: {query}')
print(f'RESPONSE: {response}')
print(f'YES/NO: {extract_yes_no(response.response)}')
print(f'GT ANSWER: {sample_elm["answer"]}')
print(f'GT LONG ANSWER: {sample_elm["long_answer"]}')

QUERY: Should all human immunodeficiency virus-infected patients with end-stage renal disease be excluded from transplantation?
RESPONSE:  Based on the context information provided, I would say no to the query. The survey results suggest that while the majority of transplant centers require HIV testing of prospective kidney recipients, only a small percentage of centers would not transplant an HIV-infected patient who is otherwise a good candidate for transplantation. Additionally, some centers have performed transplants on HIV-infected patients in the past. These findings suggest that there is a possibility of transplanting HIV-infected patients with end-stage renal disease, and therefore, it is not necessary to exclude all HIV-infected patients with end-stage renal disease from transplantation.
YES/NO: no
GT ANSWER: ['no']
GT LONG ANSWER: The great majority of U.S. renal transplant centers will not transplant kidneys to HIV-infected patients with end-stage renal disease, even if thei

#### [OPTIONAL] [Ragas](https://docs.ragas.io/en/stable/index.html) evaluation
Following are the commonly used metrics for evaluating a RAG workflow:
* [Faithfulness](https://docs.ragas.io/en/stable/concepts/metrics/faithfulness.html): Measures the factual correctness of the generated answer based on the retrived context. Value lies between 0 and 1. **Evaluated using a LLM.**
* [Answer Relevance](https://docs.ragas.io/en/stable/concepts/metrics/answer_relevance.html): Measures how relevant the answer is to the given query. Value lies between 0 and 1. **Evaluated using a LLM.**
* [Context Precision](https://docs.ragas.io/en/stable/concepts/metrics/context_precision.html): Precision of the retriever as measured using the retrieved and the ground truth context. Value lies between 0 and 1.

Additional metrics can be used based on the use-case:
* [Context Relevancy](https://docs.ragas.io/en/stable/concepts/metrics/context_relevancy.html)
* [Context Recall](https://docs.ragas.io/en/stable/concepts/metrics/context_recall.html) (requires ground truth answer)
* [Answer semantic similarity](https://docs.ragas.io/en/stable/concepts/metrics/semantic_similarity.html) (requires ground truth answer)
* [Answer Correctness](https://docs.ragas.io/en/stable/concepts/metrics/answer_correctness.html) (requires ground truth answer)

In [21]:
retrieved_nodes = query_engine.retriever.retrieve(query)

In [22]:
eval_data = {
    "question": [query],
    "answer": [response.response],
    "contexts": [[node.text for node in retrieved_nodes]],
    "ground_truths": [[sample_elm['long_answer']]],
    }
pprint(eval_data)

{'answer': [' Based on the context information provided, I would say no to the '
            'query. The survey results suggest that while the majority of '
            'transplant centers require HIV testing of prospective kidney '
            'recipients, only a small percentage of centers would not '
            'transplant an HIV-infected patient who is otherwise a good '
            'candidate for transplantation. Additionally, some centers have '
            'performed transplants on HIV-infected patients in the past. These '
            'findings suggest that there is a possibility of transplanting '
            'HIV-infected patients with end-stage renal disease, and '
            'therefore, it is not necessary to exclude all HIV-infected '
            'patients with end-stage renal disease from transplantation.'],
 'contexts': [['Human immunodeficiency virus (HIV)-infected patients have '
               'generally been excluded from transplantation. Recent advances '
        

In [24]:
eval_obj = RagasEval(
            metrics=["faithfulness", "relevancy", "precision"], 
            eval_llm_type=rag_cfg["eval_llm_type"], eval_llm_name=rag_cfg["eval_llm_name"]
            )
eval_result = eval_obj.evaluate(eval_data)
print(eval_result)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`


Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

Exception in thread Thread-8:
Traceback (most recent call last):
  File "/pkgs/python-3.10.12/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/fs01/projects/aieng/public/rag_bootcamp/envs/rag_pubmed_qa/lib/python3.10/site-packages/ragas/executor.py", line 75, in run
    results = self.loop.run_until_complete(self._aresults())
  File "/pkgs/python-3.10.12/lib/python3.10/asyncio/base_events.py", line 649, in run_until_complete
    return future.result()
  File "/fs01/projects/aieng/public/rag_bootcamp/envs/rag_pubmed_qa/lib/python3.10/site-packages/ragas/executor.py", line 63, in _aresults
    raise e
  File "/fs01/projects/aieng/public/rag_bootcamp/envs/rag_pubmed_qa/lib/python3.10/site-packages/ragas/executor.py", line 58, in _aresults
    r = await future
  File "/pkgs/python-3.10.12/lib/python3.10/asyncio/tasks.py", line 571, in _wait_for_one
    return f.result()  # May raise f.exception().
  File "/fs01/projects/aieng/public/rag_bootcamp/envs/rag

ExceptionInRunner: The runner thread which was running the jobs raised an exeception. Read the traceback above to debug it. You can also pass `raise_exceptions=False` incase you want to show only a warning message instead.

### 5.1 - Dense Search
Set *hybrid_search_alpha* to 1.0 for dense vector search.

In [25]:
rag_cfg["hybrid_search_alpha"] = 1.0

In [26]:
# Recreate query engine
query_engine_args = set_query_engine_args(rag_cfg, docs)
pprint(query_engine_args)
query_engine = RAGQueryEngine(
    retriever_type=rag_cfg['retriever_type'], vector_index=index, llm_model_name=rag_cfg['llm_name']).create(**query_engine_args)

# Get response
response = query_engine.query(query)

# Print response
print(f'QUERY: {query}')
print(f'RESPONSE: {response}')
print(f'YES/NO: {extract_yes_no(response.response)}')
print(f'GT ANSWER: {sample_elm["answer"]}')
print(f'GT LONG ANSWER: {sample_elm["long_answer"]}')

{'hybrid_search_alpha': 1.0,
 'query_mode': 'hybrid',
 'response_mode': 'compact',
 'similarity_top_k': 5,
 'use_reranker': False}
QUERY: Should all human immunodeficiency virus-infected patients with end-stage renal disease be excluded from transplantation?
RESPONSE:  Based on the context information provided, I would say no to the query. The survey results showed that while the majority of transplant centers require HIV testing of prospective kidney recipients and would not transplant an individual who refuses HIV testing, there are some centers that would consider transplanting an HIV-infected patient. Additionally, the study on elderly kidneys found that there were no differences in survival rates between elderly and younger recipients. These findings suggest that HIV-infected patients with end-stage renal disease should not be automatically excluded from transplantation.
YES/NO: no
GT ANSWER: ['no']
GT LONG ANSWER: The great majority of U.S. renal transplant centers will not trans

#### [OPTIONAL] Ragas evaluation

In [27]:
retrieved_nodes = query_engine.retriever.retrieve(query)

eval_data = {
    "question": [query],
    "answer": [response.response],
    "contexts": [[node.text for node in retrieved_nodes]],
    "ground_truths": [[sample_elm['long_answer']]],
    }

eval_result = eval_obj.evaluate(eval_data)
print(eval_result)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`


Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

Exception in thread Thread-9:
Traceback (most recent call last):
  File "/pkgs/python-3.10.12/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/fs01/projects/aieng/public/rag_bootcamp/envs/rag_pubmed_qa/lib/python3.10/site-packages/ragas/executor.py", line 75, in run
    results = self.loop.run_until_complete(self._aresults())
  File "/pkgs/python-3.10.12/lib/python3.10/asyncio/base_events.py", line 649, in run_until_complete
    return future.result()
  File "/fs01/projects/aieng/public/rag_bootcamp/envs/rag_pubmed_qa/lib/python3.10/site-packages/ragas/executor.py", line 63, in _aresults
    raise e
  File "/fs01/projects/aieng/public/rag_bootcamp/envs/rag_pubmed_qa/lib/python3.10/site-packages/ragas/executor.py", line 58, in _aresults
    r = await future
  File "/pkgs/python-3.10.12/lib/python3.10/asyncio/tasks.py", line 571, in _wait_for_one
    return f.result()  # May raise f.exception().
  File "/fs01/projects/aieng/public/rag_bootcamp/envs/rag

ExceptionInRunner: The runner thread which was running the jobs raised an exeception. Read the traceback above to debug it. You can also pass `raise_exceptions=False` incase you want to show only a warning message instead.

### 5.2 - Hybrid Search
Set *hybrid_search_alpha* to 0.5 for hybrid search with equal weightage for dense and sparse (keyword-based) search.

In [ ]:
rag_cfg["hybrid_search_alpha"] = 0.5

In [ ]:
# Recreate query engine
query_engine_args = set_query_engine_args(rag_cfg, docs)
pprint(query_engine_args)
query_engine = RAGQueryEngine(
    retriever_type=rag_cfg['retriever_type'], vector_index=index, llm_model_name=rag_cfg['llm_name']).create(**query_engine_args)

# Get response
response = query_engine.query(query)

# Print response
print(f'QUERY: {query}')
print(f'RESPONSE: {response}')
print(f'YES/NO: {extract_yes_no(response.response)}')
print(f'GT ANSWER: {sample_elm["answer"]}')
print(f'GT LONG ANSWER: {sample_elm["long_answer"]}')

#### [OPTIONAL] Ragas evaluation

In [ ]:
retrieved_nodes = query_engine.retriever.retrieve(query)

eval_data = {
    "question": [query],
    "answer": [response.response],
    "contexts": [[node.text for node in retrieved_nodes]],
    "ground_truths": [[sample_elm['long_answer']]],
    }

eval_result = eval_obj.evaluate(eval_data)
print(eval_result)

### 5.3 - Using Re-ranker
Set *use_reranker* to *True* to re-rank the context after retrieving it from the vector database.

In [ ]:
rag_cfg["use_reranker"] = True
rag_cfg["hybrid_search_alpha"] = 1.0 # Using dense search

In [ ]:
# Recreate query engine
query_engine_args = set_query_engine_args(rag_cfg, docs)
pprint(query_engine_args)
query_engine = RAGQueryEngine(
    retriever_type=rag_cfg['retriever_type'], vector_index=index, llm_model_name=rag_cfg['llm_name']).create(**query_engine_args)

# Get response
response = query_engine.query(query)

# Print response
print(f'QUERY: {query}')
print(f'RESPONSE: {response}')
print(f'YES/NO: {extract_yes_no(response.response)}')
print(f'GT ANSWER: {sample_elm["answer"]}')
print(f'GT LONG ANSWER: {sample_elm["long_answer"]}')

#### [OPTIONAL] Ragas evaluation

In [ ]:
retrieved_nodes = query_engine.retriever.retrieve(query)

eval_data = {
    "question": [query],
    "answer": [response.response],
    "contexts": [[node.text for node in retrieved_nodes]],
    "ground_truths": [[sample_elm['long_answer']]],
    }

eval_result = eval_obj.evaluate(eval_data)
print(eval_result)